In [1]:
import cv2
import numpy as np
from collections import deque
import math
import time

# --- 1. Módulo de Conteo de Dedos (Reciclado del Ejercicio 10) ---
def count_fingers(hand_contour):
    if len(hand_contour) < 5:
        return 0
        
    hull = cv2.convexHull(hand_contour, returnPoints=False)
    if len(hull) < 3:
        return 0
        
    defects = cv2.convexityDefects(hand_contour, hull)
    if defects is None:
        return 0

    finger_count = 0
    for i in range(defects.shape[0]):
        s, e, f, d = defects[i, 0]
        start = tuple(hand_contour[s][0])
        end = tuple(hand_contour[e][0])
        far = tuple(hand_contour[f][0])

        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
        
        # Ignorar posibles errores matemáticos
        if (2 * b * c) == 0: continue
        angle_rad = math.acos((b**2 + c**2 - a**2) / (2 * b * c))
        angle_deg = math.degrees(angle_rad)

        if angle_deg < 90 and d > 20000:
            finger_count += 1
            
    # El número de dedos suele ser el número de valles + 1
    return finger_count + 1

# --- 2. Configuración Inicial ---

# Puntos para el modo dibujo
drawing_points = deque(maxlen=512)

# Variables de estado para los comandos
background_color_index = 0
background_colors = [(20, 20, 20), (128, 0, 0), (0, 128, 0)] # Gris oscuro, Azul, Verde
previous_gesture = -1
last_command_time = 0

# Definir rango de color de piel en HSV
lower_skin = np.array([0, 48, 80], dtype=np.uint8)
upper_skin = np.array([20, 255, 255], dtype=np.uint8)

# --- 3. Bucle Principal ---

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

print("Asistente de Gestos iniciado. Presiona 'q' para salir.")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)

    # Creamos un lienzo o "panel de control" para mostrar info
    h, w, _ = frame.shape
    canvas = np.full((h, w, 3), background_colors[background_color_index], dtype=np.uint8)
    
    # Definimos una Región de Interés (ROI) para la mano
    roi = frame[100:400, 350:600]
    cv2.rectangle(frame, (350, 100), (600, 400), (0, 255, 0), 2)
    
    # --- Detección de Manos ---
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    current_gesture = 0
    hand_detected = False

    if len(contours) > 0:
        hand_contour = max(contours, key=cv2.contourArea)
        
        # Solo procesar si el contorno es suficientemente grande
        if cv2.contourArea(hand_contour) > 5000:
            hand_detected = True
            cv2.drawContours(roi, [hand_contour], -1, (0, 0, 255), 3)
            
            # --- Conteo de Dedos ---
            current_gesture = count_fingers(hand_contour)
            
            # Lógica para el modo dibujo (Gesto 1)
            if current_gesture == 1:
                M = cv2.moments(hand_contour)
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                # Mapeamos el centro de la ROI al frame completo
                global_center = (center[0] + 350, center[1] + 100)
                drawing_points.appendleft(global_center)

    # --- Lógica de Comandos por Gestos (Gestión de Estado) ---
    # Solo ejecutamos un comando si el gesto ha cambiado y ha pasado un tiempo
    if current_gesture != previous_gesture and (time.time() - last_command_time) > 1.0:
        command_text = ""
        if current_gesture == 2:
            command_text = "Comando: Cambiar Color Fondo"
            background_color_index = (background_color_index + 1) % len(background_colors)
            last_command_time = time.time()
        elif current_gesture == 3:
            command_text = "Comando: Limpiar Pizarra"
            drawing_points.clear()
            last_command_time = time.time()
        elif current_gesture == 5:
            command_text = "Comando: Captura de Pantalla"
            filename = f"captura_{time.strftime('%Y%m%d_%H%M%S')}.jpg"
            cv2.imwrite(filename, frame)
            print(f"Guardado: {filename}")
            last_command_time = time.time()
        
        if command_text:
             cv2.putText(canvas, command_text, (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        
        previous_gesture = current_gesture

    # --- Dibujar y Mostrar Información ---
    
    # Dibujar la trayectoria si hay puntos
    for i in range(1, len(drawing_points)):
        if drawing_points[i - 1] is not None and drawing_points[i] is not None:
            cv2.line(frame, drawing_points[i - 1], drawing_points[i], (255, 0, 0), 4)

    # Combinar el frame con el lienzo
    alpha = 0.4 # Transparencia del video
    combined_display = cv2.addWeighted(canvas, 1 - alpha, frame, alpha, 0)

    # Mostrar estado actual
    status_text = f"Dedos: {current_gesture}" if hand_detected else "Mano no detectada"
    cv2.putText(combined_display, status_text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Lista de Comandos
    cv2.putText(combined_display, "Comandos:", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
    cv2.putText(combined_display, "1 Dedo: Dibujar", (20, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(combined_display, "2 Dedos: Cambiar Fondo", (20, 160), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(combined_display, "3 Dedos: Limpiar Pizarra", (20, 190), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(combined_display, "5 Dedos: Captura", (20, 220), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Asistente de Gestos", combined_display)
    # cv2.imshow("Mascara", mask) # Descomenta para depurar la detección

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- 4. Liberar Recursos ---
cap.release()
cv2.destroyAllWindows()

Asistente de Gestos iniciado. Presiona 'q' para salir.
Guardado: captura_20251017_170532.jpg
Guardado: captura_20251017_170534.jpg
Guardado: captura_20251017_170535.jpg
Guardado: captura_20251017_170546.jpg
Guardado: captura_20251017_170600.jpg
Guardado: captura_20251017_170604.jpg
Guardado: captura_20251017_170639.jpg
